# 学习Chroma

In [1]:
import os

# 获取当前目录
current_directory = os.getcwd()

# 设置persist_directory为Advanced_RAG_From_Scratch文件夹
persist_directory = os.path.join(current_directory, '..', 'ChromaVDB')
print(persist_directory)

d:\VSCode\VSCode仓库\RAG\Advanced_RAG_From_Scratch\tests\..\ChromaVDB


In [2]:
import os
from dotenv import load_dotenv # type: ignore

# 设置代理
os.environ["http_proxy"] = "127.0.0.1:7890"
os.environ["https_proxy"] = "127.0.0.1:7890"

# 加载环境变量
load_dotenv()

True

In [3]:
api_key = os.getenv('ZETATECHS_API_KEY')
base_url = os.getenv('ZETATECHS_API_BASE')

In [4]:
from langchain_openai import OpenAIEmbeddings # type: ignore

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key, base_url=base_url)


# 1. 初始化

In [5]:
from langchain_chroma import Chroma # type: ignore

# 获取当前目录
current_directory = os.getcwd()
# 设置persist_directory为Advanced_RAG_From_Scratch文件夹
persist_directory = os.path.join(current_directory, '..', 'ChromaVDB')

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory=persist_directory,  # Where to save data locally, remove if not necessary
)

# 2. 添加新的文档

In [9]:
from langchain.document_loaders import PyPDFLoader # type: ignore
from langchain.text_splitter import RecursiveCharacterTextSplitter # type: ignore

file_path = "../files/UnderstandingDeepLearning-ZH-CN-240721.pdf"

loader = PyPDFLoader(file_path) # 创建 PyPDFLoader 实例
documents = loader.load() # 加载 PDF 文件并转换为文本数据
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap  = 100)
documents_chunks = splitter.split_documents(documents)

In [10]:
documents_chunks

[Document(metadata={'source': '../files/UnderstandingDeepLearning-ZH-CN-240721.pdf', 'page': 0}, page_content='理解深度学习\nSimon J.D. Prince\nFebruary 15, 2024'),
 Document(metadata={'source': '../files/UnderstandingDeepLearning-ZH-CN-240721.pdf', 'page': 1}, page_content='ii'),
 Document(metadata={'source': '../files/UnderstandingDeepLearning-ZH-CN-240721.pdf', 'page': 2}, page_content='目录\n1介绍 1\n1.1监督学习 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\n1.1.1回归和分类问题 . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2\n1.1.2输入. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2\n1.1.3机器学习模型 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\n1.1.4深度神经网络 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.1.5结构化输出 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.2无监督学习 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.2.1生成式模型 . . . . . . . . . . . . . . . . . . . . . . . . 

In [11]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents_chunks))]

vector_store.add_documents(documents=documents_chunks, ids=uuids)

['56f74b11-7917-44d4-b51b-5dfa3d76aad5',
 '1a84c25d-1ff7-493b-8258-06aac5711ce6',
 'dfae6e63-309a-4893-bcba-31c73c2f0a86',
 '0d0b438a-ed2b-4a2b-9912-3e422bec8908',
 '7a479cbc-c2fe-468e-808c-5a49e54d283f',
 '225a6e9a-3130-4de7-8e7e-3efae9d05d78',
 'ea8bf32a-acb3-480d-8391-1f36c969dc84',
 'fc9af2cb-6d79-45b7-9d96-ee53e638db3a',
 'a093848e-595b-43be-8f97-13df3a53f109',
 'f9e17048-0bf5-4a72-820c-7cc57613fdd1',
 '2f76b11d-3ffc-4dd6-88fc-e17453523f83',
 '77204da6-3574-485a-ad11-915df4e95aaa',
 '58d33213-8317-4fdd-93c5-dab88c98a304',
 '64e91136-b6b3-4006-b0d3-cadc5375ee00',
 'e1c4ff99-13fa-4503-89ed-6593568ed11b',
 'ec3fd155-c374-462f-be74-37b61899bbd0',
 'b3c52a43-349e-4b42-8179-f55c738133e5',
 'aa3f0c6c-16b1-4e61-a53b-233da2ad4b72',
 'cd347bab-a7b1-403e-a3ca-cd27f6491ebf',
 'bf8ccee5-34c3-432c-a4bb-aa92343b2a16',
 '815f11de-4bd8-41f2-8588-fc676b0ad044',
 'e8014f40-8342-40e4-a95b-f18580835a91',
 '31a222a2-b5a7-4a2e-a65c-64a749363e57',
 '2ea5d4f6-2a65-4d3c-92e4-2e112bc01bf6',
 '164dc632-0ded-

# 3. 删除

In [ ]:
vector_store.delete(ids=uuids[-1])

# 4. 查询向量

In [12]:
query = "深度强化学习有哪些一般的方法？"

In [17]:
results = vector_store.similarity_search(
    query,
    k=2,
)
for res in results:
    print(f"* {res.page_content} \n\n**********\n\n [{res.metadata}] \n\n")

* 352 CHAPTER 19. 深度强化学习
由于最大化操作导致的状态价值系统性过高估计，并提出了双 Q学习，通过同时训练两
个模型来解决这个问题。这种方法后来被应用于深度 Q学习（Van Hasselt 等人，2016） ，
虽然其有效性后来受到了质疑（ Hessel等人，2018） 。Wang等人(2016)引入了深度对
决网络，其中网络的两个部分分别预测状态的价值和每个动作的相对优势，这种解耦可
以提高稳定性，因为有时状态的价值更为重要，而具体采取哪种动作的影响不大。
Fortunato 等人（2018）提出了噪声深度 Q-网络，其特点是 Q-网络中的部分权重乘
以噪声以增加预测的随机性并促进探索行为。该网络能够在逐步收敛到合理策略的过程
中学会减少噪声的强度。分布式 DQN（Bellemare 等人，2017a；Dabney等人，2018，继
Morimura 等人，2010）的目标是估计回报分布的更全面信息，而不仅仅是期望值。这使
得网络有可能减轻最坏情况下的影响，并且还可以通过预测更高阶矩来提高性能，因为
这为训练提供了更丰富的信号。 Rainbow（Hessel等人，2018）结合了六项对原始深度
Q-学习算法的改进，包括决策网络、分布式 DQN和噪声DQN，从而提高了在 ATARI
基准测试上的训练速度和最终性能。
策略梯度 ：Williams （1992）首次提出了 REINFORCE 算法。”策略梯度方法 ”一
词最早见于 Sutton等人（1999） 。Konda和Tsitsiklis （1999）引入了演员 -评论家算法。
使用不同的基准来降低方差在 Greensmith 等人（2004）和Peters & Schaal （2008）的
工作中被探讨。 Mei等人（2022）后来指出，价值基准主要是减少更新的激进性而非其
方差。
策略梯度已经被改进以产生确定性策略（ Silver等人，2014；Lillicrap 等人，2016；
Fujimoto 等人，2018） 。最直接的方法是对所有可能的行动进行最大化，但如果行动空
间是连续的，则每一步都需要一个优化过程。深度确定性策略梯度算法（ Lillicrap 等人，
2016）按照行为价值梯度的方向调整策略，这表明使用了演员 -评论家方法。 

**********

 [{'page':

In [22]:
results = vector_store.similarity_search_with_score(
    query, k=10
)
for res, score in results:
    # print(f"* [SIM={score:3f}] \n\n*********\n\n {res.page_content} \n\n##########\n\n [{res.metadata}] \n\n")
    print(type(score), score)

<class 'float'> 0.8206568956375122
<class 'float'> 0.8553059101104736
<class 'float'> 0.8592807054519653
<class 'float'> 0.8806620240211487
<class 'float'> 0.892906904220581
<class 'float'> 0.8954190611839294
<class 'float'> 0.8960753083229065
<class 'float'> 0.9018613696098328
<class 'float'> 0.923303484916687
<class 'float'> 0.9272459149360657


# 假如我们换一个collection_name

In [23]:
# 获取当前目录
current_directory = os.getcwd()
# 设置persist_directory为Advanced_RAG_From_Scratch文件夹
persist_directory = os.path.join(current_directory, '..', 'ChromaVDB')

vector_store = Chroma(
    collection_name="test",
    embedding_function=embeddings,
    persist_directory=persist_directory,  # Where to save data locally, remove if not necessary
)

In [25]:
query = "深度强化学习有哪些一般的方法？"

In [26]:
results = vector_store.similarity_search_with_score(
    query, k=10
)
for res, score in results:
    # print(f"* [SIM={score:3f}] \n\n*********\n\n {res.page_content} \n\n##########\n\n [{res.metadata}] \n\n")
    print(type(score), score)

# 测试写的ChromaManager 

In [1]:
import sys
sys.path.append('..')  # 会跳出tests文件夹到Advanced_RAG_From_Scratch文件夹里面找PineconeManager
from ChromaManager import ChromaVectorStoreManager

In [2]:
# 创建ChromaVectorStore实例
chroma_store = ChromaVectorStoreManager()

In [4]:
chroma_vector_store = chroma_store.get_vector_store()
type(chroma_vector_store)

langchain_chroma.vectorstores.Chroma

In [3]:
# 上传PDF文档
chroma_store.upload_pdf_file("../files/UnderstandingDeepLearning-ZH-CN-240721.pdf")

In [4]:
# 执行相似性搜索
query = "深度强化学习有哪些一般的方法？"

In [5]:
results = chroma_store.similarity_search(query)

In [7]:
for result in results:
    print(f"Content: {result['content']}\n\nMetadata: {result['metadata']}\n")

Content: 352 CHAPTER 19. 深度强化学习
由于最大化操作导致的状态价值系统性过高估计，并提出了双 Q学习，通过同时训练两
个模型来解决这个问题。这种方法后来被应用于深度 Q学习（Van Hasselt 等人，2016） ，
虽然其有效性后来受到了质疑（ Hessel等人，2018） 。Wang等人(2016)引入了深度对
决网络，其中网络的两个部分分别预测状态的价值和每个动作的相对优势，这种解耦可
以提高稳定性，因为有时状态的价值更为重要，而具体采取哪种动作的影响不大。
Fortunato 等人（2018）提出了噪声深度 Q-网络，其特点是 Q-网络中的部分权重乘
以噪声以增加预测的随机性并促进探索行为。该网络能够在逐步收敛到合理策略的过程
中学会减少噪声的强度。分布式 DQN（Bellemare 等人，2017a；Dabney等人，2018，继
Morimura 等人，2010）的目标是估计回报分布的更全面信息，而不仅仅是期望值。这使
得网络有可能减轻最坏情况下的影响，并且还可以通过预测更高阶矩来提高性能，因为
这为训练提供了更丰富的信号。 Rainbow（Hessel等人，2018）结合了六项对原始深度
Q-学习算法的改进，包括决策网络、分布式 DQN和噪声DQN，从而提高了在 ATARI
基准测试上的训练速度和最终性能。
策略梯度 ：Williams （1992）首次提出了 REINFORCE 算法。”策略梯度方法 ”一
词最早见于 Sutton等人（1999） 。Konda和Tsitsiklis （1999）引入了演员 -评论家算法。
使用不同的基准来降低方差在 Greensmith 等人（2004）和Peters & Schaal （2008）的
工作中被探讨。 Mei等人（2022）后来指出，价值基准主要是减少更新的激进性而非其
方差。
策略梯度已经被改进以产生确定性策略（ Silver等人，2014；Lillicrap 等人，2016；
Fujimoto 等人，2018） 。最直接的方法是对所有可能的行动进行最大化，但如果行动空
间是连续的，则每一步都需要一个优化过程。深度确定性策略梯度算法（ Lillicrap 等人，
2016）按照行为价值梯度的方向调整策略，这表明使用了演员 -评论家方法。

Metadata: {'page

In [8]:
# 执行带分数的相似性搜索
results_with_score = chroma_store.similarity_search_with_score(query)
for result in results_with_score:
    print(f"Content: {result['content']}\nMetadata: {result['metadata']}\nScore: {result['score']}\n")

Content: 352 CHAPTER 19. 深度强化学习
由于最大化操作导致的状态价值系统性过高估计，并提出了双 Q学习，通过同时训练两
个模型来解决这个问题。这种方法后来被应用于深度 Q学习（Van Hasselt 等人，2016） ，
虽然其有效性后来受到了质疑（ Hessel等人，2018） 。Wang等人(2016)引入了深度对
决网络，其中网络的两个部分分别预测状态的价值和每个动作的相对优势，这种解耦可
以提高稳定性，因为有时状态的价值更为重要，而具体采取哪种动作的影响不大。
Fortunato 等人（2018）提出了噪声深度 Q-网络，其特点是 Q-网络中的部分权重乘
以噪声以增加预测的随机性并促进探索行为。该网络能够在逐步收敛到合理策略的过程
中学会减少噪声的强度。分布式 DQN（Bellemare 等人，2017a；Dabney等人，2018，继
Morimura 等人，2010）的目标是估计回报分布的更全面信息，而不仅仅是期望值。这使
得网络有可能减轻最坏情况下的影响，并且还可以通过预测更高阶矩来提高性能，因为
这为训练提供了更丰富的信号。 Rainbow（Hessel等人，2018）结合了六项对原始深度
Q-学习算法的改进，包括决策网络、分布式 DQN和噪声DQN，从而提高了在 ATARI
基准测试上的训练速度和最终性能。
策略梯度 ：Williams （1992）首次提出了 REINFORCE 算法。”策略梯度方法 ”一
词最早见于 Sutton等人（1999） 。Konda和Tsitsiklis （1999）引入了演员 -评论家算法。
使用不同的基准来降低方差在 Greensmith 等人（2004）和Peters & Schaal （2008）的
工作中被探讨。 Mei等人（2022）后来指出，价值基准主要是减少更新的激进性而非其
方差。
策略梯度已经被改进以产生确定性策略（ Silver等人，2014；Lillicrap 等人，2016；
Fujimoto 等人，2018） 。最直接的方法是对所有可能的行动进行最大化，但如果行动空
间是连续的，则每一步都需要一个优化过程。深度确定性策略梯度算法（ Lillicrap 等人，
2016）按照行为价值梯度的方向调整策略，这表明使用了演员 -评论家方法。
Metadata: {'page'

### Chroma检索出来的信息是根据score升序排列的，这与pinecoe不同，需要注意一下 